In [11]:
import numpy as np

from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Activation
from keras.utils import to_categorical

from tqdm import tqdm_notebook

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train =  x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

In [5]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit(x_train, y_train, 
          batch_size=64, epochs=10,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 42s 697us/step - loss: 0.2242 - acc: 0.9319 - val_loss: 0.1034 - val_acc: 0.9660
Epoch 2/10
60000/60000 [==============================] - 40s 667us/step - loss: 0.1027 - acc: 0.9690 - val_loss: 0.0878 - val_acc: 0.9725
Epoch 3/10
60000/60000 [==============================] - 41s 678us/step - loss: 0.0735 - acc: 0.9763 - val_loss: 0.0734 - val_acc: 0.9776
Epoch 4/10
60000/60000 [==============================] - 42s 701us/step - loss: 0.0612 - acc: 0.9804 - val_loss: 0.0686 - val_acc: 0.9801
Epoch 5/10
60000/60000 [==============================] - 39s 642us/step - loss: 0.0507 - acc: 0.9837 - val_loss: 0.0677 - val_acc: 0.9797
Epoch 6/10
60000/60000 [==============================] - 28s 471us/step - loss: 0.0430 - acc: 0.9859 - val_loss: 0.0709 - val_acc: 0.9802
Epoch 7/10
60000/60000 [==============================] - 34s 561us/step - loss: 0.0394 - acc: 0.9871 - val_loss: 0.

In [7]:
model.evaluate(x_test, y_test, verbose=0)

[0.069526189794246143, 0.98229999999999995]

In [9]:
target = model.layers[1].get_weights()
u, s, v_t = np.linalg.svd(target[0], full_matrices=False)

for k in range(9):
    tmp_s = s.copy()
    tmp_s[2 ** k:] = 0
    new_weight = u @ np.diag(tmp_s) @ v_t
    
    model.layers[1].set_weights([new_weight, target[1]])
    
    print(model.evaluate(x_test, y_test, verbose=0))
model.layers[1].set_weights(target)

[10.146345379638673, 0.15939999999999999]
[4.4640375984191891, 0.16980000000000001]
[3.2330919906616211, 0.2878]
[2.8833652229309084, 0.53439999999999999]
[0.5375544863879681, 0.85270000000000001]
[0.096995719662818378, 0.97560000000000002]
[0.073573062395004399, 0.9819]
[0.069296292846260626, 0.98170000000000002]
[0.069256134514182943, 0.98199999999999998]


In [13]:
for k in tqdm_notebook(range(9)):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(2 ** k, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(x_train, y_train, 
          batch_size=64, epochs=10,
          validation_data=(x_test, y_test), verbose=0)
    print(model.evaluate(x_test, y_test, verbose=0))

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

[1.6379653503417968, 0.3574]
[1.1204903287887573, 0.59989999999999999]
[0.59145005402564998, 0.82130000000000003]
[0.29622616779208183, 0.91300000000000003]
[0.1685811268031597, 0.94789999999999996]
[0.10343013823218644, 0.96679999999999999]
[0.07126743512176327, 0.97750000000000004]
[0.072578562841452365, 0.9798]
[0.070283059116308874, 0.98170000000000002]

